In [1]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle
from statsmodels.tsa.stattools import adfuller

In [2]:
from aiagentbase import AIAgent,Controller,Memory,Perception,Actor

importing Jupyter notebook from aiagentbase.ipynb


### Rule-based strategies as agents
See ruleagents_dev: tested with backtest. Also with tradeserver. TBD: common rewards format
Note: only works for local strategies (not remote: TBD).

In [11]:
class RuleAgent(AIAgent):
    def __init__(self):
        self.agent=True
        super().__init__()
        self.actor.call_model=self.call_model
    def set_alt_data(self,alt_data_func,remote=False):
        if remote: self.gdata=anvil.server.call(alt_data_func)['gdata']
        else: self.gdata=alt_data_func()['gdata']
    def check_entry_batch(self,dfD):
        return self.act(('entry',dfD))
    def save_func(self,episode_state):
        return ()
    def check_exit_batch(self,dfD,posf):
        def exit_fn(row):
            return self.act(('exit',row,dfD[row.ticker]))
        posf['to_exit']=posf.apply(exit_fn,axis=1).values
        return posf
    def exit_predicate(self,row,df):
        return self.act(('exit',row,df))
    def Check(strat,dfD):
        return strat.check_entry_batch(dfD)
    def Exit(strat,dfD,posf):
        return strat.check_exit_batch(dfD,posf)
    def act(self,state):
        action=super().act(state)
        return action
    def reward(self,reward):
        super().reward(reward)

In [12]:
class AdaMomCMOAgent(RuleAgent):
    # Adaptive momentum strategy using CMO and ADF for non-stationarity
    def __init__(self,high=80,low=20,mid=50):
        super(AdaMomCMOAgent,self).__init__()
        self.logL=[]
        self.high=high
        self.low=low
        self.mid=mid
        self.model_type='rule_based'
        self.data_cols=['Close_n','CMO_14','datetime']
        self.regime={}
        self.entry_val={}
        self.exit_val={}
        self.logL=[]
        self.rewL=[]
    def call_model(self,state):
        # super().act(state)
        if state[0]=='entry': return self.check_entry(state[1])
        elif state[0]=='exit': return self.exit_func(state[1],state[2])
    def check_entry(self,dfD):
        # return always_buy(dfD)
        timenow=[dfD[t].iloc[-1]['datetime'] for t in dfD][0]
        hour,minute=timenow.hour,timenow.minute
        decisionsD={t:0 for t in dfD}
        stopD={t:5 for t in dfD}
        targetD={t:5 for t in dfD}
        if hour==9 and minute<=35: return decisionsD,stopD,targetD
        dataD={}
        log_entry={}
        high=self.high
        low=self.low
        mid=self.mid
        for t in dfD.keys():
            data=dfD[t]
            row=dfD[t].iloc[-1]
            if data.shape[0]>=65:
                self.logL+=[(t,timenow,data['Close_n'])]
                adf=adfuller(data['Close_n'],maxlag=30,autolag=None)
                if adf[0]>adf[4]['1%']: self.regime[t]='tr'
                else: self.regime[t]='mr'
                regime=self.regime[t]
                if regime=='tr' and row['CMO_14']>low and row['CMO_14']<mid: decisionsD[t]=1
                elif regime=='tr' and row['CMO_14']<-low and row['CMO_14']>-mid: decisionsD[t]=-1
                elif regime=='mr' and row['CMO_14']>high: decisionsD[t]=-1
                elif regime=='mr' and row['CMO_14']<-high: decisionsD[t]=1
                else: decisionsD[t]=0
                self.entry_val[t]=row['CMO_14']
                self.exit_val[t]='not_set'
        return decisionsD,stopD,targetD
    def exit_func(self,row,df):
        # return False
        data=df
        dfrow=df.iloc[-1]
        high=self.high
        low=self.low
        mid=self.mid
        regime=self.regime[row['ticker']]
        self.exit_val[row['ticker']]=dfrow['CMO_14']
        if regime=='tr' and row['quant']>0 and dfrow['CMO_14']>high: return True
        elif regime=='tr' and row['quant']<0 and dfrow['CMO_14']<-high: return True
        elif regime=='mr' and row['quant']>0 and dfrow['CMO_14']>-low: return True
        elif regime=='mr' and row['quant']<0 and dfrow['CMO_14']<low: return True
        # exit cases for detecting a trade on incorrect trend direction
        # elif regime=='tr' and row['quant']>0 and dfrow['CMO_14']<=-mid: return True
        # elif regime=='tr' and row['quant']<0 and dfrow['CMO_14']>=mid: return True
        else: return False
    def save_func(self,episode_state):
        ticker=[t for t in episode_state][0]
        return ticker,self.entry_val[ticker],self.exit_val[ticker]
    def reward(self,reward):
        self.rewL+=[reward]
        super().reward(reward)

In [8]:
class GapBetAgent(RuleAgent):
    def __init__(self,entry=50,exit=75,direction=-1):
        super(GapBetAgent,self).__init__()
        self.entry=entry
        self.exit=exit
        self.data_cols=['CMO_14','datetime']
        self.model_type='rule-based'
        # self.direction=direction
    def act(self,state):
        if state[0]=='entry': return self.check_entry(state[1])
        elif state[0]=='exit': return self.exit_func(state[1],state[2])
    def check_entry(self,dfD):
        decisionsD={t:0 for t in dfD}
        stopD={t:0.25 for t in dfD}
        targetD={t:2 for t in dfD}
        timenow=[dfD[t].iloc[-1]['datetime'] for t in dfD][0]
        date=timenow.strftime('%d-%b-%Y')
        gdir=global_direction(self.gdata[date][0])
        if abs(gdir)>.5: self.direction=1
        else: self.direction=-1
        hour,minute=timenow.hour,timenow.minute
        if hour>9 or (hour==9 and minute>35): return decisionsD,stopD,targetD
        for t in dfD:
            row=dfD[t].iloc[-1]
            if row['CMO_14']>self.entry: decisionsD[t]=self.direction
            elif row['CMO_14']<-self.entry: decisionsD[t]=-self.direction
        return decisionsD,stopD,targetD
    def exit_func(self,row,posf):
        return False

In [9]:
def do_nothing(dfD):
    empty={t:0 for t in dfD}
    return empty,empty,empty
def always_buy(dfD):
    buy={t:1 for t in dfD}
    empty={t:0 for t in dfD}
    return buy,empty,empty
def always_sell(dfD):
    sell={t:-1 for t in dfD}
    empty={t:0 for t in dfD}
    return sell,empty,empty

In [10]:
def global_direction(gdata):
    global_tickers=['^NYA','LSEG.L','^IXIC']
    direction={}
    for g in global_tickers:
        direction[g]=gdata['Close_'+g]-gdata['Open_'+g]
    return 100*sum([direction[k] for k in direction])/len(direction)
def domestic_direction(gdata):
    tickers=['^NSEI']
    direction={}
    for g in tickers:
        direction[g]=gdata['Close_'+g]-gdata['Open_'+g]
    return 100*sum([direction[k] for k in direction])/len(direction)